코드 링크 : https://github.com/qkqkfldis1/kaggle_book_bengaliai/blob/main/train_code_bengaliai_notebook.ipynb

    - 시작 단계
      • 대회에 참가할때는 반드시 평가 방법을 숙지해야 한다.
        왜냐하면 평가 방법(지표)에 따라 문제 해결 전략이 달라지기 때문이다.

# **모델 전처리**

## **요약**

제공되는 파일:
train.csv\
train_image_data_0.parquet\
train_image_data_1.parquet\
train_image_data_2.parquet\
train_image_data_3.parquet

In [ ]:
!pip install iterative-stratification

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import joblib
from tqdm import tqdm

data_dir = "../input/"
files_train = [f"train_image_data_{fid}.parquet" for fid in range(4)]
F = os.path.join(data_dir, files_train[0])
df_train_images = pd.read_parquet(F)

#이미지 파일(pd 데이터프레임)을 피클(pkl)데이터로 저장
for fname in files_train:
    F = os.path.join(data_dir, fname)
    df_train_images = pd.read_parquet(F)
    img_ids = df_train_images["image_id"].values # 샘플 번호
    img_array = df_train_images.iloc[:, 1:].values # 이미지 데이터 (137*236)
    for idx in tqdm(range(len(df_train_images))):
        img_id = img_ids[idx]
        img = img_array[idx]
        joblib.dump(img, f"../input/train_images/{img_id}.pkl")

In [ ]:
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

df_train = pd.read_csv('../input/train.csv')  # 훈련 데이터를 불러온다.

# StratifiedKFold를 위해 이미지 id와 target 값을 나눈다.
X = df_train['image_id'] #.values로 넘파이 배열로 변환 가능.
y = df_train[["grapheme_root", "vowel_diacritic", "consonant_diacritic"]]

# MultilabelStratifiedKFold로 3개의 레이블을 5개의 폴드로 나눈다.
mskf = MultilabelStratifiedKFold(n_splits=5, random_state=42, shuffle=True)
df_train["fold"] = -1
for i, (trn_idx, vld_idx) in enumerate(mskf.split(X, y)):
    df_train.loc[vld_idx, "fold"] = i
df_train["fold"] = df_train["fold"].astype(int)

# 랜덤하게 섞인 폴드가 추가된 파일을 저장
df_train.to_csv("../input/df_folds.csv", index=False)

# 다시 그 파일을 불러오고
df_folds = pd.read_csv("../input/df_folds.csv")

#훈련폴드는 80% 검증 폴드는 20% 사용.
trn_fold = [0,1,2,3]
vld_fold = [4]
trn_idx = df_folds.loc[df_folds["fold"].isin(trn_fold)].index
vld_idx = df_folds.loc[df_folds["fold"].isin(vld_fold)].index

trn_df = df_folds.loc[trn_idx]
vld_df = df_folds.loc[vld_idx]

In [ ]:
#TF DATA API
import tensorflow as tf

img_height = 137
img_width = 236
image_size = 224
batch_size = 32

def preprocess_img(img_id, label, is_training=True):
    img_file = tf.io.read_file(f"../input/train_images/{img_id}.pkl")
    img = tf.io.decode_raw(img_file, out_type=tf.uint8)
    img = tf.reshape(img, [img_height, img_width, 1])
    img = 255 - img  # 흑백 반전
    img = tf.image.grayscale_to_rgb(img)  # 그레이스케일 이미지를 RGB로 변환

    # 증강을 위한 함수 (Albumentations 대신 TensorFlow 함수 사용)
    def augment(img):
        img = tf.image.random_crop(img, size=[image_size, image_size, 3])
        img = tf.image.random_flip_left_right(img)
        # 추가적인 증강 기능을 여기에 구현 가능.
        return img

    img = augment(img) if is_training else tf.image.resize(img, [image_size,
                                                                 image_size])
    return img, label

def get_dataset(df, is_training=True):
    img_ids = df['image_id'].values
    labels = df[['grapheme_root', 'vowel_diacritic',
                 'consonant_diacritic']].values

    dataset = tf.data.Dataset.from_tensor_slices((img_ids, labels))
    dataset = dataset.map(lambda img_id, label:
                          preprocess_img(img_id, label,is_training),
                          num_parallel_calls=tf.data.AUTOTUNE)

    if is_training:
        dataset = dataset.shuffle(1024).repeat()
    return dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

trn_dataset = get_dataset(trn_df, is_training=True)
vld_dataset = get_dataset(vld_df, is_training=False)

## **전체 과정**

**1. 파일 불러오고 탐색하기**


In [ ]:
data_dir = "../input/"
files_train = [f"train_image_data_{fid}.parquet" for fid in range(4)]
->  ['train_image_data_0.parquet',
    'train_image_data_1.parquet',
    'train_image_data_2.parquet',
    'train_image_data_3.parquet']

F = os.path.join(data_dir, files_train[0])

df_train_images = pd.read_parquet(F) #parquet 확장자 파일을 읽어들이면 pd로 불러옴
df_train_images.head()


#이미지를 몇 개 불러와서 탐색한다.
idx = 0
img = df_train_images.iloc[idx, 1:].values.astype(np.uint8)
# img = 255 - img # 이미지 색 반전

plt.imshow(img.reshape(137, 236), cmap="gray")  # HEIGHT, WIDTH 순서
plt.show()

**2. Target의 분포 탐색하고 폴드 만들기**

In [ ]:
3개의 target 데이터를 불러오고 bar를 plot해서 모든 target이 적절하게 분포되어 있는지 확인한다.

df_train = pd.read_csv('../input/train.csv')

plt.figure(figsize=(20, 10))
df_train["grapheme_root"].value_counts().sort_index().plot.bar()

plt.figure(figsize=(20, 10))
df_train["vowel_diacritic"].value_counts().sort_index().plot.bar()

plt.figure(figsize=(20, 10))
df_train["consonant_diacritic"].value_counts().sort_index().plot.bar()


이때 클래스별 분산이 균일하지 않다면 Stratified Fold
를 사용할 수 있고, 만약 Target Label이 여러개라면 iterstrat라이브러리의
ml_stratifiers.MultilabelStratifiedKFold 를 사용 할 수 있다.

아래와 같이 처음에 폴드를 만들고 폴드를 추가한 pd 데이터프레임을
csv로 저장해두면 처음 한 번만 만들고 계속 재사용할 수 있다.

from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

#grapheme_root, vowel_diacritic, consonant_diacritic이 Target 데이터임.
X = df_train[["image_id", "grapheme_root", "vowel_diacritic", "consonant_diacritic"]].values[:, 0]
y = df_train[["image_id", "grapheme_root", "vowel_diacritic", "consonant_diacritic"]].values[:, 1:]

mskf = MultilabelStratifiedKFold(n_splits=5, random_state=42, shuffle=True)
df_train["fold"] = -1
for i, (trn_idx, vld_idx) in enumerate(mskf.split(X, y)):
    df_train.loc[vld_idx, "fold"] = i
df_train["fold"] = df_train["fold"].astype(int)

df_train.to_csv("../input/df_folds.csv", index=False)

아래와 같이 섞고 fold 번호를 할당한다.
fold는 원본 데이터셋의 레이블 비율을 반영하여 분할된다.
  image_id  grapheme_root  vowel_diacritic  consonant_diacritic  fold
0  Train_0             15                9                    5     2
1  Train_1            159                0                    0     3
2  Train_2             22                3                    5     1
3  Train_3             53                2                    2     0
4  Train_4             71                9                    5     4

**3. 데이터 파일 전처리**
데이터가 너무 많은 경우에 이를 판다스 데이터프레임으로 읽어들이면\
시간 손해가 발생하므로 넘파이 배열이나 pickle 파일 형태로 바꿔서 저장하고\
읽는 방식을 취하면 모델 학습 시간을 단축시킬 수 있다.

예를 들면 아래와 같이 간단하게 할 수 있다.

In [ ]:
import pandas as pd

# 데이터프레임 생성
df = pd.DataFrame({'a': [1, 2, 3], 'b': [4, 5, 6]})
# Pickle 파일로 저장
df.to_pickle('filename.pkl')

# Pickle 파일 로드
df_loaded = pd.read_pickle('filename.pkl')
# 로드된 데이터 확인
print(df_loaded)


#실제 코드
import joblib
from tqdm import tqdm

for fname in files_train:
    F = os.path.join(data_dir, fname)
    df_train_images = pd.read_parquet(F)
    img_ids = df_train_images["image_id"].values # 샘플 번호
    img_array = df_train_images.iloc[:, 1:].values # 이미지 데이터 (137*236)
    for idx in tqdm(range(len(df_train_images))):
        img_id = img_ids[idx]
        img = img_array[idx]
        joblib.dump(img, f"../input/train_images/{img_id}.pkl")


tqdm 이란? 실시간 진행상황을 나타내주는 라이브러리

from tqdm import tqdm
import time
for i in tqdm(range(10)):
    time.sleep(1)
    print(i)
 10%|█         | 1/10 [00:01<00:09,  1.00s/it]0
 20%|██        | 2/10 [00:02<00:08,  1.00s/it]1
 30%|███       | 3/10 [00:03<00:07,  1.00s/it]2
 40%|████      | 4/10 [00:04<00:06,  1.00s/it]3
 50%|█████     | 5/10 [00:05<00:05,  1.00s/it]4
 60%|██████    | 6/10 [00:06<00:04,  1.00s/it]5
 70%|███████   | 7/10 [00:07<00:03,  1.00s/it]6
 80%|████████  | 8/10 [00:08<00:02,  1.00s/it]7
 90%|█████████ | 9/10 [00:09<00:01,  1.00s/it]8
100%|██████████| 10/10 [00:10<00:00,  1.00s/it]9


joblib 이란? 대규모 데이터 구조를 빠르게 저장하고 불러오는 라이브러리
사이킷런 모델을 저장하거나 불러올때도 사용된다..
import joblib
모델 저장: joblib.dump(model, 'model_filename.pkl')
모델 불러오기: model = joblib.load('model_filename.pkl')

또한 병렬 처리를 지원한다. (Parallel , delayed 메서드 사용)
from joblib import Parallel, delayed
results = Parallel(n_jobs=-1)(delayed(my_function)(i) for i in my_list)

Parallel = n_jobs를 사용해 cpu 코어 사용 개수 할당
delayed = 함수 실행을 지연시켜 병렬처리 하게 만듬
즉, 위 코드는 my_list에 있는 인수를 모두 my_function에 넣으며 실행시키는 것을
병렬로 처리하는 것을 구현한 예시임.

# 모델을 훈련할때 n_jobs = -1 하는것도 내부적으로 joblib를 알아서 구현하는것임.

**4. 훈련, 검증 세트 만들기**


In [ ]:
df_train = pd.read_csv("../input/train.csv")
df_train["fold"] = pd.read_csv("../input/df_folds.csv")["fold"]

trn_fold = [0,1,2,3]
vld_fold = [4]
trn_idx = df_train.loc[df_train["fold"].isin(trn_fold)].index
vld_idx = df_train.loc[df_train["fold"].isin(vld_fold)].index

trn_df = df_train.loc[trn_idx]
vld_df = df_train.loc[vld_idx]

사전 훈련된 모델을 사용하기 위해 일자로 늘여서 벡터로 저장한 이미지를
3차원(RGB)로 변환해야 한다.

csv = trn_df.reset_index()
img_ids = csv["image_id"].values
img_height = 137
img_width = 236

index = 0
img_id = img_ids[index]
img = joblib.load(f"../input/train_images/{img_id}.pkl")
img = img.reshape(img_height, img_width).astype(np.uint8)
img = 255 - img # 흑백 반전이 성능이 더 좋았음.
img = img[:, :, np.newaxis]
img = np.repeat(img, 3, 2) # img, repeat, axis

#확인
plt.imshow(img)
plt.show()


**5. 데이터 증강**
albumentations는 특히 이미지 데이터를 증강시켜주는 강력한 라이브러리이다.\
물론 어떤 데이터 증강 기법을 사용할지,
얼마나 할지도 하이퍼파라미터 이므로 많은 실험을 통해 효과적인 방법을 찾아야 한다.

In [ ]:


import albumentations as A
image_size = 224
train_transform = A.Compose(
    [
        A.RandomResizedCrop(height=image_size, width=image_size),
        A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=0.05, p=0.5),
        A.RGBShift(r_shift_limit=0.05, g_shift_limit=0.05, b_shift_limit=0.05, p=0.5),
        A.RandomBrightnessContrast(p=0.5),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ]
)

valid_transfrom = A.Compose(
    [
        A.Resize(height=image_size, width=image_size),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ]
)


**6. 훈련**


In [ ]:
Label을 넘파이 배열로 읽어들이고.. (val_df도 포함)
label_1 = trn_df.iloc[index].grapheme_root
label_2 = trn_df.iloc[index].vowel_diacritic
label_3 = trn_df.iloc[index].consonant_diacritic

# **모델 훈련**

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.callbacks import ReduceLROnPlateau

# 모델 불러오기 및 마지막 레이어 교체
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
x = Dense(186, activation='softmax')(x)
model = tf.keras.Model(inputs = base_model.input, outputs = x)

# 모델 컴파일
model.compile(optimizer = Adam(lr=0.001),
              loss = CategoricalCrossentropy(),
              metrics = ['accuracy'])

# 스케줄러 설정
scheduler = ReduceLROnPlateau(
    monitor='val_accuracy',
    factor=0.5,
    patience=7,
    verbose=1,
    mode='max',
    min_lr=0.0001
)

# 모델을 GPU로 설정 (사용 가능한 경우)
model = model.cuda() if tf.test.is_gpu_available() else model

**CUT-MIX (데이터 증강)**


In [ ]:
def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

for inputs, targets in tqdm(trn_loader):
    inputs = inputs.cuda()
    targets = targets.cuda()
    break

lam = np.random.beta(1.0, 1.0)
rand_index = tf.random.shuffle(inputs.size()[0])
targets_gra = targets[:, 0]
targets_vow = targets[:, 1]
targets_con = targets[:, 2]
shuffled_targets_gra = targets_gra[rand_index]
shuffled_targets_vow = targets_vow[rand_index]
shuffled_targets_con = targets_con[rand_index]

bbx1, bby1, bbx2, bby2 = rand_bbox(inputs.size(), lam)
inputs[:, :, bbx1:bbx2, bby1:bby2] = inputs[rand_index, :, bbx1:bbx2, bby1:bby2]
lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (inputs.size()[-1] * inputs.size()[-2]))
plt.imshow(inputs[10].permute(1, 2, 0).cpu())
plt.show()

**Mix-up (데이터 증강)**


In [ ]:
# lam: Lambda
lam = np.random.beta(1.0, 1.0)
print(lam)
rand_index = tf.random.shuffle(inputs.size()[0])

shuffled_targets_gra = targets_gra[rand_index]
shuffled_targets_vow = targets_vow[rand_index]
shuffled_targets_con = targets_con[rand_index]

batch_size = inputs.size()[0]
index = tf.random.shuffle(batch_size)

inputs = lam * inputs + (1 - lam) * inputs[index, :]

plt.imshow(inputs[10].permute(1, 2, 0).detach().cpu())
plt.show()

**Stochastic Weight Averaging (SWA)**   
딥 러닝에서 모델 가중치의 평균을 사용해 단일 모델의 성능을\
향상 시키는 앙상블 기법
swa_start_epoch는 일반적으로 전체 훈련 에포크의 중간 지점 이후로 설정하는 것이 좋다.


In [ ]:
# SWA 콜백 정의
class SWA(Callback):
    def __init__(self, swa_epoch):
        super(SWA, self).__init__()
        self.swa_epoch = swa_epoch
        self.swa_weights = None

    def on_epoch_end(self, epoch, logs=None):
        if epoch == self.swa_epoch:
            self.swa_weights = self.model.get_weights()
        elif epoch > self.swa_epoch:
            current_weights = self.model.get_weights()
            n = epoch - self.swa_epoch + 1
            for i in range(len(current_weights)):
                self.swa_weights[i] = (self.swa_weights[i] * (n - 1) + current_weights[i]) / n

# 모델 훈련
swa_callback = SWA(swa_epoch=5)  # SWA를 5번째 에폭 이후부터 적용
model.fit(x_train, y_train, epochs=10, callbacks=[swa_callback])

# SWA 평균 가중치로 모델 설정
model.set_weights(swa_callback.swa_weights)